In [1]:
import os
import numpy as np
import pandas as pd
print(os.getcwd())

c:\DATA\Google Drive\KU\Semester 3\Cognitive Science 3\Project\cs3-vr\ipynb


In [2]:
conditions = pd.read_csv("../data/conditions.csv")
paper_scores = pd.read_csv("../data/exam/paper_test_scores.csv")
exam_scores = pd.read_csv("../data/exam/exam-results.tsv", sep = "\t")
paper_scores.rename(columns = {"participant": "participant_number"}, inplace="True")

In [3]:
def grade_declarative(participant):
    result = np.zeros((11), dtype=int)
    result[0] = participant.iloc[:,1].values[0]
    result[1] = 1 if 'and will buy' in participant.iloc[:,2].values[0] else 0
    result[2] = 1 if 'willingness and ability' in participant.iloc[:,3].values[0] else 0
    result[3] = 1 if 'can be expected to increase' in participant.iloc[:,4].values[0] else 0
    result[4] = 1 if 'can be expected to increase' in participant.iloc[:,5].values[0] else 0
    result[5] = 1 if 'households or companies' in participant.iloc[:,6].values[0] else 0
    result[6] = 1 if 'along with other products' in participant.iloc[:,7].values[0] else 0
    result[7] = 1 if 'fulfill the same need' in participant.iloc[:,8].values[0] else 0
    result[8] = 1 if 'None of the above' in participant.iloc[:,9].values[0] else 0
    result[9] = 1 if 'None of the above' in participant.iloc[:,10].values[0] else 0
    result[10] = 1 if 'are balanced' in participant.iloc[:,11].values[0] else 0
    
    return np.array([result[0], np.sum(result[1:])])
    


def grade_conceptual(participant):
    result = np.zeros((7), dtype=int)
    result[0] = participant.iloc[:,1].values[0]
    result[1] += 1 if 'a)' in participant.iloc[:,12].values[0] else 0
    result[1] += 1 if 'd)' in participant.iloc[:,12].values[0] else 0
    result[2] = 1 if 'likely to increase' in participant.iloc[:,13].values[0] else 0
    result[3] += 1 if 'Running shoes' in participant.iloc[:,14].values[0] else 0
    result[3] += 1 if 'Washers' in participant.iloc[:,14].values[0] else 0
    result[3] += 1 if 'cleats' in participant.iloc[:,14].values[0] else 0
    result[4] += 1 if 'excess supply' in participant.iloc[:,15].values[0] else 0
    result[5] += 1 if 'taxes' in participant.iloc[:,16].values[0] else 0
    result[5] += 1 if 'maintaining' in participant.iloc[:,16].values[0] else 0
    result[5] += 1 if 'buying' in participant.iloc[:,16].values[0] else 0
    result[5] += 1 if 'wages' in participant.iloc[:,16].values[0] else 0
    result[6] += 1 if 'Pens' in participant.iloc[:,17].values[0] else 0
    result[6] += 1 if 'Android' in participant.iloc[:,17].values[0] else 0
    # print(result)
    return np.array([result[0], np.sum(result[1:])])



# grade(exam_scores.iloc[[5]])

In [4]:
conceptual_score = np.array([grade_conceptual(exam_scores.iloc[[i]]) for i in range(exam_scores.shape[0])])
conceptual_score = pd.DataFrame({'participant_number': conceptual_score[:,0], 'conceptual_score': conceptual_score[:,1]})

declarative_score = np.array([grade_declarative(exam_scores.iloc[[i]]) for i in range(exam_scores.shape[0])])
declarative_score = pd.DataFrame({'participant_number': declarative_score[:,0], 'declarative_score': declarative_score[:,1]})

In [16]:
merged = pd.merge(conditions, paper_scores, on="participant_number")
merged = pd.merge(merged, conceptual_score, on="participant_number")
merged = pd.merge(merged, declarative_score, on="participant_number")
merged.drop(['q1', 'q2', 'q3', 'q4', 'q5'], axis = 1, inplace=True)
merged.rename(columns={"total": "procedural_score"}, inplace=True)
merged['total_score'] = merged.procedural_score + merged.conceptual_score + merged.declarative_score
merged = merged[~merged.participant_number.isin(['9553'])]
merged


,participant_number,condition,procedural_score,conceptual_score,declarative_score,total_score
0,7187,1,10,10,7,27
1,4121,1,7,11,7,25
2,7742,0,10,9,6,25
3,6976,0,10,7,7,24
4,5938,0,2,11,5,18
5,3947,0,0,8,5,13
6,8657,1,8,11,6,25
7,2627,0,5,5,3,13
8,4554,1,0,10,4,14
9,9322,1,2,12,5,19


In [15]:
merged[merged.condition == 1].describe()

,participant_number,condition,procedural_score,conceptual_score,declarative_score,total_score
count,10.000000,10.0,10.000000,10.000000,10.000000,10.000000
mean,6343.200000,1.0,3.900000,10.400000,5.500000,19.800000
std,1833.268108,0.0,3.900142,1.712698,1.269296,5.159673
min,3996.000000,1.0,0.000000,7.000000,3.000000,11.000000
25%,4813.500000,1.0,0.000000,10.000000,5.000000,17.250000
50%,6330.000000,1.0,4.000000,11.000000,6.000000,19.000000
75%,7304.000000,1.0,6.750000,11.750000,6.000000,24.500000
max,9322.000000,1.0,10.000000,12.000000,7.000000,27.000000


In [7]:

merged[merged.condition == 0].describe()

,participant_number,condition,procedural_score,conceptual_score,declarative_score,total_score
count,10.000000,10.0,10.0,10.000000,10.000000,10.000000
mean,4920.100000,0.0,5.0,8.700000,5.600000,19.300000
std,1882.309011,0.0,4.0,1.888562,1.349897,5.375872
min,2069.000000,0.0,0.0,5.000000,3.000000,13.000000
25%,3831.500000,0.0,2.0,7.250000,5.000000,13.750000
50%,4735.000000,0.0,5.0,9.500000,6.000000,20.000000
75%,6463.750000,0.0,9.0,10.000000,6.750000,23.500000
max,7742.000000,0.0,10.0,11.000000,7.000000,27.000000
